In [1]:
# i tutaj zaczyna się przygoda!

In [2]:
## Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
import pandas as pd
import io
from scipy import stats
#import itertools
import datetime
import statsmodels.api as sm

%run funkcje.ipynb

In [3]:
# collecting data

infections = pd.read_csv('pre11_2020_infected_data.txt', sep = "\t", encoding = 'ANSI')
deaths = pd.read_csv('pre11_2020_deaths_data.txt', sep = "\t", encoding = 'ANSI')

infections2 = pd.read_csv('post11_2020_infected_data.txt', sep = "\t", encoding = 'ANSI')
deaths2 = pd.read_csv('post11_2020_deaths_data.txt', sep = "\t", encoding = 'ANSI')

population = pd.read_csv('new_ludnosc2.txt', sep = '\t', encoding = 'utf-8')

pop_area_dens = pd.read_csv('pop_area_dens.txt', sep = '\t', encoding = 'utf-8')

beds = pd.read_csv('beds_10k.txt', sep = '\t', encoding = 'utf-8')

In [4]:
# editing and sorting dataframes

pop_area_dens_cities = pop_area_dens.loc[
    (pop_area_dens['district'].str[0].str.isupper()) | (pop_area_dens['district'] == "m.st.Warszawa")]
pop_area_dens_noncities = pop_area_dens.loc[
    (pop_area_dens['district'].str[0].str.islower()) & (pop_area_dens['district'] != "m.st.Warszawa")]

infections_cities1 = infections.loc[infections['district'].str.contains("m.", regex=False)]
infections_cities2 = infections2.loc[infections2['district'].str.contains("m.", regex=False)]

deaths_cities1 = deaths.loc[deaths['district'].str.contains("m.", regex=False)]
deaths_cities2 = deaths2.loc[deaths2['district'].str.contains("m.", regex=False)]

population['new_district'] = population['district'].apply(edit_pop)
population['new_province'] = population['province'].apply(edit_pop)
population = population.drop(columns=['district'])
population = population.drop(columns=['province'])
population = population.rename(columns = {'new_district':'district'})
population = population.rename(columns = {'new_province':'province'})
population['district'] = population['district'].str.lower()
population['province'] = population['province'].str.lower()
#population = population.sort_values('district')

beds['new_district'] = beds['district'].apply(edit_inf)
beds['new_province'] = beds['province'].apply(edit_inf)
beds = beds.drop(columns=['district'])
beds = beds.drop(columns=['province'])
beds = beds.rename(columns = {'new_district':'district'})
beds = beds.rename(columns = {'new_province':'province'})
beds['district'] = beds['district'].str.lower()
beds['province'] = beds['province'].str.lower()

# tutaj edit nowych danych
pop_area_dens['new_district'] = pop_area_dens['district'].apply(edit_new)
pop_area_dens['new_province'] = pop_area_dens['province'].apply(edit_new)
pop_area_dens = pop_area_dens.drop(columns=['district'])
pop_area_dens = pop_area_dens.drop(columns=['province'])
pop_area_dens = pop_area_dens.rename(columns = {'new_district':'district'})
pop_area_dens = pop_area_dens.rename(columns = {'new_province':'province'})
pop_area_dens['district'] = pop_area_dens['district'].str.lower()
pop_area_dens['province'] = pop_area_dens['province'].str.lower()

infections['date'] = pd.to_datetime(infections['date'], format = '%d.%m.%Y') 
infections['district'] = infections['district'].apply(edit_inf)
infections['province'] = infections['province'].apply(edit_inf)
infections['district'] = infections['district'].str.lower()
infections['province'] = infections['province'].str.lower()
infections = infections[infections.district != infections.province]
infections = infections.sort_values(by=['province', 'district', 'date'])

infections_cities1['date'] = pd.to_datetime(infections_cities1['date'], format = '%d.%m.%Y') 
infections_cities1['district'] = infections_cities1['district'].apply(edit_inf)
infections_cities1['province'] = infections_cities1['province'].apply(edit_inf)
infections_cities1['district'] = infections_cities1['district'].str.lower()
infections_cities1['province'] = infections_cities1['province'].str.lower()
infections_cities1 = infections_cities1[infections_cities1.district != infections_cities1.province]
infections_cities1 = infections_cities1.sort_values(by=['province', 'district', 'date'])

infections2['date'] = pd.to_datetime(infections2['date'], format = '%d.%m.%Y') 
infections2['district'] = infections2['district'].apply(edit_inf)
infections2['province'] = infections2['province'].apply(edit_inf)
infections2['district'] = infections2['district'].str.lower()
infections2['province'] = infections2['province'].str.lower()
infections2 = infections2[infections2.district != 'sumazpowiatów']
infections2 = infections2[infections2.district != infections2.province]
infections2 = infections2.sort_values(by=['district', 'date'])

infections_cities2['date'] = pd.to_datetime(infections_cities2['date'], format = '%d.%m.%Y') 
infections_cities2['district'] = infections_cities2['district'].apply(edit_inf)
infections_cities2['province'] = infections_cities2['province'].apply(edit_inf)
infections_cities2['district'] = infections_cities2['district'].str.lower()
infections_cities2['province'] = infections_cities2['province'].str.lower()
infections_cities2 = infections_cities2[infections_cities2.district != 'sumazpowiatów']
infections_cities2 = infections_cities2[infections_cities2.district != infections_cities2.province]
infections_cities2 = infections_cities2.sort_values(by=['district', 'date'])

deaths['date'] = pd.to_datetime(deaths['date'], format = '%d.%m.%Y') 
deaths['district'] = deaths['district'].apply(edit_inf)
deaths['province'] = deaths['province'].apply(edit_inf)
deaths['district'] = deaths['district'].str.lower()
deaths['province'] = deaths['province'].str.lower()
deaths = deaths[deaths.district != deaths.province] #nie uwzględniamy województw, można zmienić, tylko, że nie mam w df danych
deaths = deaths.sort_values(by=['district', 'date']) #gęstości zaludnienia w całych województwach

deaths_cities1['date'] = pd.to_datetime(deaths_cities1['date'], format = '%d.%m.%Y') 
deaths_cities1['district'] = deaths_cities1['district'].apply(edit_inf)
deaths_cities1['province'] = deaths_cities1['province'].apply(edit_inf)
deaths_cities1['district'] = deaths_cities1['district'].str.lower()
deaths_cities1['province'] = deaths_cities1['province'].str.lower()
deaths_cities1 = deaths_cities1[deaths_cities1.district != deaths_cities1.province] #nie uwzględniamy województw, można zmienić, tylko, że nie mam w df danych
deaths_cities1 = deaths_cities1.sort_values(by=['district', 'date']) #gęstości zaludnienia w całych województwach

deaths2['date'] = pd.to_datetime(deaths2['date'], format = '%d.%m.%Y') 
deaths2['district'] = deaths2['district'].apply(edit_inf)
deaths2['province'] = deaths2['province'].apply(edit_inf)
deaths2['district'] = deaths2['district'].str.lower()
deaths2['province'] = deaths2['province'].str.lower()
deaths2 = deaths2[deaths2.district != 'sumazpowiatów']
deaths2 = deaths2[deaths2.district != deaths2.province]
deaths2 = deaths2.sort_values(by=['district', 'date'])

deaths_cities2['date'] = pd.to_datetime(deaths_cities2['date'], format = '%d.%m.%Y') 
deaths_cities2['district'] = deaths_cities2['district'].apply(edit_inf)
deaths_cities2['province'] = deaths_cities2['province'].apply(edit_inf)
deaths_cities2['district'] = deaths_cities2['district'].str.lower()
deaths_cities2['province'] = deaths_cities2['province'].str.lower()
deaths_cities2 = deaths_cities2[deaths_cities2.district != 'sumazpowiatów']
deaths_cities2 = deaths_cities2[deaths_cities2.district != deaths_cities2.province]
deaths_cities2 = deaths_cities2.sort_values(by=['district', 'date'])



In [5]:
print(beds)
print(pop_area_dens)

     beds_10k        district            province
0       21.25   bolesławiecki        dolnośląskie
1       20.27  dzierżoniowski        dolnośląskie
2       22.17       głogowski        dolnośląskie
3       26.49        górowski        dolnośląskie
4       16.22        jaworski        dolnośląskie
..        ...             ...                 ...
357     62.56         wałecki  zachodniopomorskie
358      6.61         łobeski  zachodniopomorskie
359     69.43        koszalin  zachodniopomorskie
360     87.46        szczecin  zachodniopomorskie
361     28.10     świnoujście  zachodniopomorskie

[362 rows x 3 columns]
     area_ha  population  density_ppl_per_ha        district  \
0     130351       90103            0.691234   bolesławiecki   
1      47851      100813            2.106811  dzierżoniowski   
2      44327       89102            2.010107       głogowski   
3      73827       34870            0.472320        górowski   
4      58155       50116            0.861766        jawo

In [6]:
infections['population'] = 0
infections_cities1['population'] = 0
deaths['population'] = 0
deaths_cities1['population'] = 0
infections2['population'] = 0
infections_cities2['population'] = 0
deaths2['population'] = 0
deaths_cities2['population'] = 0

infections['area_ha'] = 0
infections_cities1['area_ha'] = 0
deaths['area_ha'] = 0
deaths_cities1['area_ha'] = 0
infections2['area_ha'] = 0
infections_cities2['area_ha'] = 0
deaths2['area_ha'] = 0
deaths_cities2['area_ha'] = 0

infections['density_ppl_per_ha'] = 0
infections_cities1['density_ppl_per_ha'] = 0
deaths['density_ppl_per_ha'] = 0
deaths_cities1['density_ppl_per_ha'] = 0
infections2['density_ppl_per_ha'] = 0
infections_cities2['density_ppl_per_ha'] = 0
deaths2['density_ppl_per_ha'] = 0
deaths_cities2['density_ppl_per_ha'] = 0

# infections['beds_10k'] = 0
# infections_cities1['beds_10k'] = 0
# deaths['beds_10k'] = 0
# deaths_cities1['beds_10k'] = 0
# infections2['beds_10k'] = 0
# infections_cities2['beds_10k'] = 0
# deaths2['beds_10k'] = 0
# deaths_cities2['beds_10k'] = 0

In [7]:
# print_full(population)
# print_full(pop_area_dens)

In [8]:
# dodawanie populacji do df: infections i deaths + przepisywanie ostatniej valid wartości + kolumna new_infeced, new_death
last_infections = []
last_infections_cit = []
last_deaths = []
last_deaths_cit = []

# for ind in population.index:
#     inf_cond = (infections['district'] == population['district'][ind]) & (infections['province'] == population['province'][ind])
#     death_cond = (deaths['district'] == population['district'][ind]) & (deaths['province'] == population['province'][ind])
#     infections.loc[inf_cond, 'population'] = population['population'][ind]
#     deaths.loc[death_cond, 'population'] = population['population'][ind]
    
#     inf_last_valid_idx = infections.loc[inf_cond].apply(pd.Series.last_valid_index)[3]
#     death_last_valid_idx = deaths.loc[death_cond].apply(pd.Series.last_valid_index)[3]
    
#     last_infections.append(infections.loc[inf_cond, 'infected_total'].loc[inf_last_valid_idx])
#     last_deaths.append(deaths.loc[death_cond, 'dead_total'].loc[death_last_valid_idx])
    
#     infections.loc[inf_cond, 'new_infected'] = infections.loc[inf_cond, 'infected_total'].diff()
#     deaths.loc[death_cond, 'new_death'] = deaths.loc[death_cond, 'dead_total'].diff()
    
for ind in pop_area_dens.index:
    inf_cond = (infections['district'] == pop_area_dens['district'][ind]) & (infections['province'] == pop_area_dens['province'][ind])
    inf_cit_cond = (infections_cities1['district'] == pop_area_dens['district'][ind]) & (infections_cities1['province'] == pop_area_dens['province'][ind])
    death_cond = (deaths['district'] == pop_area_dens['district'][ind]) & (deaths['province'] == pop_area_dens['province'][ind])
    death_cit_cond = (deaths_cities1['district'] == pop_area_dens['district'][ind]) & (deaths_cities1['province'] == pop_area_dens['province'][ind])
    infections.loc[inf_cond, 'population'] = pop_area_dens['population'][ind]
    infections.loc[inf_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    infections.loc[inf_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    infections_cities1.loc[inf_cit_cond, 'population'] = pop_area_dens['population'][ind]
    infections_cities1.loc[inf_cit_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    infections_cities1.loc[inf_cit_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    deaths.loc[death_cond, 'population'] = pop_area_dens['population'][ind]
    deaths.loc[death_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    deaths.loc[death_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    deaths_cities1.loc[death_cit_cond, 'population'] = pop_area_dens['population'][ind]
    deaths_cities1.loc[death_cit_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    deaths_cities1.loc[death_cit_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    
    inf_last_valid_idx = infections.loc[inf_cond].apply(pd.Series.last_valid_index)[3]
    inf_cit_last_valid_idx = infections_cities1.loc[inf_cit_cond].apply(pd.Series.last_valid_index)[3]
    death_last_valid_idx = deaths.loc[death_cond].apply(pd.Series.last_valid_index)[3]
    death_cit_last_valid_idx = deaths_cities1.loc[death_cit_cond].apply(pd.Series.last_valid_index)[3]
    
    last_infections.append(infections.loc[inf_cond, 'infected_total'].loc[inf_last_valid_idx])
    if not np.isnan(inf_cit_last_valid_idx):
        last_infections_cit.append(infections_cities1.loc[inf_cit_cond, 'infected_total'].loc[inf_cit_last_valid_idx])
    else:
        last_infections_cit.append(0)
    last_deaths.append(deaths.loc[death_cond, 'dead_total'].loc[death_last_valid_idx])
    if not np.isnan(death_cit_last_valid_idx):
        last_deaths_cit.append(deaths_cities1.loc[death_cit_cond, 'dead_total'].loc[death_cit_last_valid_idx])
    else:
        last_deaths_cit.append(0)
    
    infections.loc[inf_cond, 'new_infected'] = infections.loc[inf_cond, 'infected_total'].diff()
    infections_cities1.loc[inf_cit_cond, 'new_infected'] = infections_cities1.loc[inf_cit_cond, 'infected_total'].diff()
    deaths.loc[death_cond, 'new_death'] = deaths.loc[death_cond, 'dead_total'].diff()
    deaths_cities1.loc[death_cit_cond, 'new_death'] = deaths_cities1.loc[death_cit_cond, 'dead_total'].diff()


In [9]:
# dodawanie liczby łóżek na 10k mieszkańców do df: infections i deaths
    
for ind in beds.index:
    inf_cond = (infections['district'] == beds['district'][ind]) & (infections['province'] == beds['province'][ind])
    inf_cit_cond = (infections_cities1['district'] == beds['district'][ind]) & (infections_cities1['province'] == beds['province'][ind])
    death_cond = (deaths['district'] == beds['district'][ind]) & (deaths['province'] == beds['province'][ind])
    death_cit_cond = (deaths_cities1['district'] == beds['district'][ind]) & (deaths_cities1['province'] == beds['province'][ind])
    infections.loc[inf_cond, 'beds_10k'] = beds['beds_10k'][ind]
    infections_cities1.loc[inf_cit_cond, 'beds_10k'] = beds['beds_10k'][ind]
    deaths.loc[death_cond, 'beds_10k'] = beds['beds_10k'][ind]
    deaths_cities1.loc[death_cit_cond, 'beds_10k'] = beds['beds_10k'][ind]

In [10]:
# tutaj probujemy zrobic cumsum poprawnie no i dodaje populację do dataframe'u jak wyżej

# for ind in population.index:
#     inf_cond = (infections2['district'] == population['district'][ind]) & (infections2['province'] == population['province'][ind])
#     death_cond = (deaths2['district'] == population['district'][ind]) & (deaths2['province'] == population['province'][ind])
    
#     infections2.loc[inf_cond, 'population'] = population['population'][ind]
#     deaths2.loc[death_cond, 'population'] = population['population'][ind]
    
#     infections2.loc[inf_cond, 'infected_total'] = infections2.loc[inf_cond, 'new_infected'].cumsum()
#     deaths2.loc[death_cond, 'dead_total'] = deaths2.loc[death_cond, 'new_death'].cumsum()
    
#     infections2.loc[inf_cond, 'infected_total'] = infections2.loc[inf_cond, 'infected_total'] + last_infections[ind]
#     deaths2.loc[death_cond, 'dead_total'] = deaths2.loc[death_cond, 'dead_total'] + last_deaths[ind]


for ind in pop_area_dens.index:
    inf_cond = (infections2['district'] == pop_area_dens['district'][ind]) & (infections2['province'] == pop_area_dens['province'][ind])
    inf_cit_cond = (infections_cities2['district'] == pop_area_dens['district'][ind]) & (infections_cities2['province'] == pop_area_dens['province'][ind])
    death_cond = (deaths2['district'] == pop_area_dens['district'][ind]) & (deaths2['province'] == pop_area_dens['province'][ind])
    death_cit_cond = (deaths_cities2['district'] == pop_area_dens['district'][ind]) & (deaths_cities2['province'] == pop_area_dens['province'][ind])
    
    infections2.loc[inf_cond, 'population'] = pop_area_dens['population'][ind]
    infections2.loc[inf_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    infections2.loc[inf_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    infections_cities2.loc[inf_cit_cond, 'population'] = pop_area_dens['population'][ind]
    infections_cities2.loc[inf_cit_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    infections_cities2.loc[inf_cit_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    deaths2.loc[death_cond, 'population'] = pop_area_dens['population'][ind]
    deaths2.loc[death_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    deaths2.loc[death_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    deaths_cities2.loc[death_cit_cond, 'population'] = pop_area_dens['population'][ind]
    deaths_cities2.loc[death_cit_cond, 'area_ha'] = pop_area_dens['area_ha'][ind]
    deaths_cities2.loc[death_cit_cond, 'density_ppl_per_ha'] = pop_area_dens['density_ppl_per_ha'][ind]
    
    
    infections2.loc[inf_cond, 'infected_total'] = infections2.loc[inf_cond, 'new_infected'].cumsum()
    infections_cities2.loc[inf_cit_cond, 'infected_total'] = infections_cities2.loc[inf_cit_cond, 'new_infected'].cumsum()
    deaths2.loc[death_cond, 'dead_total'] = deaths2.loc[death_cond, 'new_death'].cumsum()
    deaths_cities2.loc[death_cit_cond, 'dead_total'] = deaths_cities2.loc[death_cit_cond, 'new_death'].cumsum()
    
    infections2.loc[inf_cond, 'infected_total'] = infections2.loc[inf_cond, 'infected_total'] + last_infections[ind]
    infections_cities2.loc[inf_cit_cond, 'infected_total'] = infections_cities2.loc[inf_cit_cond, 'infected_total'] + last_infections[ind]
    deaths2.loc[death_cond, 'dead_total'] = deaths2.loc[death_cond, 'dead_total'] + last_deaths[ind]
    deaths_cities2.loc[death_cit_cond, 'dead_total'] = deaths_cities2.loc[death_cit_cond, 'dead_total'] + last_deaths[ind]
    
#print(infections2.loc[infections2['date'] == "2020-11-24"])

In [11]:
# dodawanie liczby łóżek na 10k mieszkańców do df: infections i deaths
    
for ind in beds.index:
    inf_cond = (infections2['district'] == beds['district'][ind]) & (infections2['province'] == beds['province'][ind])
    inf_cit_cond = (infections_cities2['district'] == beds['district'][ind]) & (infections_cities2['province'] == beds['province'][ind])
    death_cond = (deaths2['district'] == beds['district'][ind]) & (deaths2['province'] == beds['province'][ind])
    death_cit_cond = (deaths_cities2['district'] == beds['district'][ind]) & (deaths_cities2['province'] == beds['province'][ind])
    infections2.loc[inf_cond, 'beds_10k'] = beds['beds_10k'][ind]
    infections_cities2.loc[inf_cit_cond, 'beds_10k'] = beds['beds_10k'][ind]
    deaths2.loc[death_cond, 'beds_10k'] = beds['beds_10k'][ind]
    deaths_cities2.loc[death_cit_cond, 'beds_10k'] = beds['beds_10k'][ind]

In [12]:
# dodawanie liczby łóżek na 10k mieszkańców do df pop_area_dens
    
for ind in beds.index:
    pop_area_dens_cities
    cond = (pop_area_dens['district'] == beds['district'][ind]) & (pop_area_dens['province'] == beds['province'][ind])
    cond_cit = (pop_area_dens_cities['district'] == beds['district'][ind]) & (pop_area_dens_cities['province'] == beds['province'][ind])
    cond_noncit = (pop_area_dens_noncities['district'] == beds['district'][ind]) & (pop_area_dens_noncities['province'] == beds['province'][ind])
    pop_area_dens.loc[cond, 'beds_10k'] = beds['beds_10k'][ind]
    pop_area_dens_cities.loc[cond_cit, 'beds_10k'] = beds['beds_10k'][ind]
    pop_area_dens_noncities.loc[cond_noncit, 'beds_10k'] = beds['beds_10k'][ind]

In [13]:
#łączę dataframe'y
# infections = pd.concat([infections, infections2], verify_integrity=True)
# deaths = pd.concat([deaths, deaths2], verify_integrity=True)
# infections = pd.concat([infections, infections2], ignore_index=True)
# deaths = pd.concat([deaths, deaths2], ignore_index=True)
infections = pd.concat([infections, infections2])
infections = infections.sort_values(by=['province', 'district', 'date'])
infections = infections.reset_index(drop=True)

infections_cities = pd.concat([infections_cities1, infections_cities2])
infections_cities = infections_cities.sort_values(by=['province', 'district', 'date'])
infections_cities = infections_cities.reset_index(drop=True)

deaths = pd.concat([deaths, deaths2])
deaths = deaths.sort_values(by=['province', 'district', 'date'])
deaths = deaths.reset_index(drop=True)

deaths_cities = pd.concat([deaths_cities1, deaths_cities2])
deaths_cities = deaths_cities.sort_values(by=['province', 'district', 'date'])
deaths_cities = deaths_cities.reset_index(drop=True)



In [14]:
#   daty dla infekcji
#   04.03.2020 - 23.11.2020
#   24.11.2020 - 06.02.2022

#   daty dla smierci
#   04.03.2020 - 23.11.2020
#   24.11.2020 - 17.02.2022
# print(deaths)
#print(deaths.loc[deaths['district'] == "augustowski"])
#print(infections.loc[infections['district'] == "augustowski"])
#print(infections.loc[infections['date'] == '2022-01-06'])
#print_full(deaths.loc[deaths['district'] == "aleksandrowski"])
#print(pop_area_dens)
#print(population)
# print(infections.loc[infections['date'] == '2020-11-23'])
# print(infections.loc[infections['date'] == '2020-11-24'])
# print_full(infections.loc[infections['district'] == "leski"])

# t = infections.loc[infections['date'] == '2020-11-23', 'district'].tolist()
# s = infections.loc[infections['date'] == '2021-11-24', 'district'].tolist()
# print(compare(s, t))
#print(pop_area_dens)
# print(deaths.loc[deaths['date'] == '2020-11-23'])
# print(deaths.loc[deaths['date'] == '2021-11-24'])
# p = deaths.loc[deaths['date'] == '2020-11-23', 'district'].tolist()
# r = deaths.loc[deaths['date'] == '2021-11-24', 'district'].tolist()
# print(compare(p, r))

# zbadam zaraz dla róznych dat tak samo infekcje i śmierci

# ok, raczej reszta git
# problem opisany na discordzie #dane
print_full(infections.loc[infections.district == "warszawa"])

            date     province  district  infected_total  population  area_ha  \
87063 2020-03-04  mazowieckie  warszawa             0.0     1790658    51724   
87064 2020-03-05  mazowieckie  warszawa             0.0     1790658    51724   
87065 2020-03-06  mazowieckie  warszawa             0.0     1790658    51724   
87066 2020-03-07  mazowieckie  warszawa             0.0     1790658    51724   
87067 2020-03-08  mazowieckie  warszawa             0.0     1790658    51724   
87068 2020-03-09  mazowieckie  warszawa             0.0     1790658    51724   
87069 2020-03-10  mazowieckie  warszawa             0.0     1790658    51724   
87070 2020-03-11  mazowieckie  warszawa             0.0     1790658    51724   
87071 2020-03-12  mazowieckie  warszawa             0.0     1790658    51724   
87072 2020-03-13  mazowieckie  warszawa             9.0     1790658    51724   
87073 2020-03-14  mazowieckie  warszawa            13.0     1790658    51724   
87074 2020-03-15  mazowieckie  warszawa 

In [15]:
# tworzenie listy nr. indeksów dla pierwszego dnia pandemii dla danego powiatu dla zakażeń
infections['pandemic_day'] = 0
infections['infected_mean'] = 0
infections['new_infected_mean'] = 0
infections['infected_in_wave'] = infections['infected_total']
infections['scnd_wave_day'] = 0
window_size_total = 7
window_size_new = 14
min_value_inf = 10  # dokładnie to liczymy od kiedy ta wartość zostanie przekroczona
first_day_inf = [] 
#t = []
for ind in population.index:
    cond = (infections['district'] == population['district'][ind]) & (infections['province'] == population['province'][ind])
    one_district = infections.loc[cond]
    if not one_district.empty:
        #t.append(population['district'][ind])
        one_district = one_district.sort_values(by=['date'])
        weeks = one_district['infected_total'].rolling(window_size_total, center=True)
        weeks_mean = weeks.mean()
        x_days = one_district['new_infected'].rolling(window_size_new, center=True)
        x_days_mean = x_days.mean()
        condition = one_district['infected_total'] > min_value_inf
        indices = one_district.index[condition]
        indices_list = indices.tolist()
        if indices_list:
            first_day_inf.append(indices_list[0])
        else:
            indices_list[0] = one_district.index.to_list()[-1]
            first_day_inf.append(indices_list[0])
        
        infections.loc[cond, 'pandemic_day'] = infections.index[cond] - indices_list[0]
        infections.loc[cond, 'infected_mean'] = weeks_mean
        infections.loc[cond, 'new_infected_mean'] = x_days_mean
        infections.loc[cond, 'new_infected_mean_per_capita'] = x_days_mean / infections.loc[cond, 'population']
        
        cond1 = (one_district['date'] >= "2020-12-10")
        cond2 = (one_district['date'] <= "2021-04-01")
        min_idx = infections.loc[cond & cond1 & cond2, "new_infected_mean"].idxmin()
        cond4 = (infections.index >= min_idx)
        infections.loc[cond & cond4, 'infected_in_wave'] = infections.loc[cond & cond4, 'infected_total'] - infections.iloc[min_idx, 3]
        infections.loc[cond, 'infected_in_wave_mean'] = infections.loc[cond, 'infected_mean']
        infections.loc[cond & cond4, 'infected_in_wave_mean'] = infections.loc[cond & cond4, 'infected_mean'] - infections.iloc[min_idx, 10]
        infections.loc[cond, 'scnd_wave_day'] = infections.loc[cond, 'pandemic_day'] - infections.iloc[min_idx, 9]

In [16]:
# print_full(infections.loc[infections.district == "warszawa", ['infected_mean', 'infected_in_wave_mean']])

In [17]:
# tworzenie listy nr. indeksów dla pierwszego dnia pandemii dla danego powiatu dla zakażeń w miastach
infections_cities['pandemic_day'] = 0
infections_cities['infected_mean'] = 0
infections_cities['new_infected_mean'] = 0
infections_cities['infected_in_wave'] = infections_cities['infected_total']
infections_cities['infected_in_wave_mean'] = infections_cities['infected_mean']
infections_cities['scnd_wave_day'] = 0
window_size_total = 7
window_size_new = 14
min_value_inf = 10  # dokładnie to liczymy od kiedy ta wartość zostanie przekroczona
first_day_inf = [] 
#t = []
for ind in population.index:
    cond = (infections_cities['district'] == population['district'][ind]) & (infections_cities['province'] == population['province'][ind])
    one_district = infections_cities.loc[cond]
    if not one_district.empty:
        #t.append(population['district'][ind])
        one_district = one_district.sort_values(by=['date'])
        weeks = one_district['infected_total'].rolling(window_size_total, center=True)
        weeks_mean = weeks.mean()
        x_days = one_district['new_infected'].rolling(window_size_new, center=True)
        x_days_mean = x_days.mean()
        
        condition = one_district['infected_total'] > min_value_inf
        indices = one_district.index[condition]
        indices_list = indices.tolist()
        if indices_list:
            first_day_inf.append(indices_list[0])
        else:
            indices_list[0] = one_district.index.to_list()[-1]
            first_day_inf.append(indices_list[0])
        
        infections_cities.loc[cond, 'pandemic_day'] = infections_cities.index[cond] - indices_list[0]
        infections_cities.loc[cond, 'infected_mean'] = weeks_mean
        infections_cities.loc[cond, 'new_infected_mean'] = x_days_mean
        infections_cities.loc[cond, 'new_infected_mean_per_capita'] = x_days_mean / infections_cities.loc[cond, 'population']
        
        cond1 = (one_district['date'] >= "2020-11-01")
        cond2 = (one_district['date'] <= "2021-04-01")
        min_idx = infections_cities.loc[cond & cond1 & cond2, "new_infected_mean"].idxmin()
        cond4 = (infections_cities.index >= min_idx)
        infections_cities.loc[cond & cond4, 'infected_in_wave'] = infections_cities.loc[cond & cond4, 'infected_total'] - infections_cities.iloc[min_idx, 3]
        infections_cities.loc[cond, 'infected_in_wave_mean'] = infections_cities.loc[cond, 'infected_mean']
        infections_cities.loc[cond & cond4, 'infected_in_wave_mean'] = infections_cities.loc[cond & cond4, 'infected_mean'] - infections_cities.iloc[min_idx, 10]
        infections_cities.loc[cond, 'scnd_wave_day'] = infections_cities.loc[cond, 'pandemic_day'] - infections_cities.iloc[min_idx, 9]

In [18]:
# tworzenie listy nr. indeksów dla pierwszego dnia pandemii dla danego powiatu dla zgonów
deaths['pandemic_day'] = 0
deaths['dead_mean'] = 0
deaths['new_death_mean'] = 0
deaths['dead_in_wave'] = deaths['dead_total']
deaths['dead_in_wave_mean'] = deaths['dead_mean']
deaths['scnd_wave_day'] = 0
window_size_total = 7
window_size_new = 14
min_value_deaths = 3 # dokładnie to liczymy od kiedy ta wartość zostanie przekroczona
first_day_deaths = []
#t = []
for ind in population.index:
    cond = (deaths['district'] == population['district'][ind]) & (deaths['province'] == population['province'][ind])
    one_district = deaths.loc[cond]
    if not one_district.empty:
        #t.append(population['district'][ind])
        one_district = one_district.sort_values(by=['date'])
        weeks = one_district['dead_total'].rolling(window_size_total, center=True)
        weeks_mean = weeks.mean()
        x_days = one_district['new_death'].rolling(window_size_new, center=True)
        x_days_mean = x_days.mean()
       
        condition = one_district['dead_total'] > min_value_deaths
        indices = one_district.index[condition]
        indices_list = indices.tolist()
        if indices_list:
            first_index = indices_list[0]
            first_day_deaths.append(first_index)
        else:
            first_index = one_district.index.to_list()[-1]
            first_day_deaths.append(first_index)

        deaths.loc[cond, 'pandemic_day'] = deaths.index[cond] - first_index
        deaths.loc[cond, 'dead_mean'] = weeks_mean
        deaths.loc[cond, 'new_death_mean'] = x_days_mean
        deaths.loc[cond, 'new_death_mean_per_capita'] = x_days_mean / deaths.loc[cond, 'population']

        cond1 = (one_district['date'] >= "2020-11-01")
        cond2 = (one_district['date'] <= "2021-04-01")
        min_idx = deaths.loc[cond & cond1 & cond2, "new_death_mean"].idxmin()
        cond4 = (deaths.index >= min_idx)
        deaths.loc[cond & cond4, 'dead_in_wave'] = deaths.loc[cond & cond4, 'dead_total'] - deaths.iloc[min_idx, 3]
        deaths.loc[cond, 'dead_in_wave_mean'] = deaths.loc[cond, 'dead_mean']
        deaths.loc[cond & cond4, 'dead_in_wave_mean'] = deaths.loc[cond & cond4, 'dead_mean'] - deaths.iloc[min_idx, 10]
        deaths.loc[cond, 'scnd_wave_day'] = deaths.loc[cond, 'pandemic_day'] - deaths.iloc[min_idx, 9]

In [19]:
# tworzenie listy nr. indeksów dla pierwszego dnia pandemii dla danego powiatu dla zgonów w miastach
deaths_cities['pandemic_day'] = 0
deaths_cities['dead_mean'] = 0
deaths_cities['new_death_mean'] = 0
deaths_cities['dead_in_wave'] = deaths_cities['dead_total']
deaths_cities['dead_in_wave_mean'] = deaths_cities['dead_mean']
deaths_cities['scnd_wave_day'] = 0
window_size_total = 7
window_size_new = 14
min_value_deaths = 3 # dokładnie to liczymy od kiedy ta wartość zostanie przekroczona
first_day_deaths = []
#t = []
for ind in population.index:
    cond = (deaths_cities['district'] == population['district'][ind]) & (deaths_cities['province'] == population['province'][ind])
    one_district = deaths_cities.loc[cond]
    if not one_district.empty:
        #t.append(population['district'][ind])
        one_district = one_district.sort_values(by=['date'])
        weeks = one_district['dead_total'].rolling(window_size_total, center=True)
        weeks_mean = weeks.mean()
        x_days = one_district['new_death'].rolling(window_size_new, center=True)
        x_days_mean = x_days.mean()
        
        condition = one_district['dead_total'] > min_value_deaths
        indices = one_district.index[condition]
        indices_list = indices.tolist()
        if indices_list:
            first_index = indices_list[0]
            first_day_deaths.append(first_index)
        else:
            first_index = one_district.index.to_list()[-1]
            first_day_deaths.append(first_index)

        deaths_cities.loc[cond, 'pandemic_day'] = deaths_cities.index[cond] - first_index
        deaths_cities.loc[cond, 'dead_mean'] = weeks_mean
        deaths_cities.loc[cond, 'new_death_mean'] = x_days_mean
        deaths_cities.loc[cond, 'new_death_mean_per_capita'] = x_days_mean / deaths_cities.loc[cond, 'population']
        
        cond1 = (one_district['date'] >= "2020-11-01")
        cond2 = (one_district['date'] <= "2021-04-01")
        min_idx = deaths_cities.loc[cond & cond1 & cond2, "new_death_mean"].idxmin()
        cond4 = (deaths_cities.index >= min_idx)
        deaths_cities.loc[cond & cond4, 'dead_in_wave'] = deaths_cities.loc[cond & cond4, 'dead_total'] - deaths_cities.iloc[min_idx, 3]
        deaths_cities.loc[cond, 'dead_in_wave_mean'] = deaths_cities.loc[cond, 'dead_mean']
        deaths_cities.loc[cond & cond4, 'dead_in_wave_mean'] = deaths_cities.loc[cond & cond4, 'dead_mean'] - deaths_cities.iloc[min_idx, 10]
        deaths_cities.loc[cond, 'scnd_wave_day'] = deaths_cities.loc[cond, 'pandemic_day'] - deaths_cities.iloc[min_idx, 9]

In [20]:
# średnia data 0. dnia pandemii
day_0_dates = infections.loc[infections.pandemic_day == 0, 'date']
day_0_date_mean = (np.array(day_0_dates, dtype='datetime64[D]')
        .view('i8')
        .mean()
        .astype('datetime64[D]'))

In [29]:
print_full(infections.loc[infections.district == "lublin"])

            date   province district  infected_total  population  area_ha  \
45166 2020-03-04  lubelskie   lublin             0.0      339784    14747   
45167 2020-03-05  lubelskie   lublin             0.0      339784    14747   
45168 2020-03-06  lubelskie   lublin             0.0      339784    14747   
45169 2020-03-07  lubelskie   lublin             0.0      339784    14747   
45170 2020-03-08  lubelskie   lublin             0.0      339784    14747   
45171 2020-03-09  lubelskie   lublin             0.0      339784    14747   
45172 2020-03-10  lubelskie   lublin             NaN      339784    14747   
45173 2020-03-11  lubelskie   lublin             NaN      339784    14747   
45174 2020-03-12  lubelskie   lublin             NaN      339784    14747   
45175 2020-03-13  lubelskie   lublin             NaN      339784    14747   
45176 2020-03-14  lubelskie   lublin             NaN      339784    14747   
45177 2020-03-15  lubelskie   lublin             NaN      339784    14747   

In [22]:
print_full(deaths.loc[deaths.district == "warszawa"])

            date     province  district  dead_total  population  area_ha  \
88784 2020-03-04  mazowieckie  warszawa         0.0     1790658    51724   
88785 2020-03-05  mazowieckie  warszawa         0.0     1790658    51724   
88786 2020-03-06  mazowieckie  warszawa         0.0     1790658    51724   
88787 2020-03-07  mazowieckie  warszawa         0.0     1790658    51724   
88788 2020-03-08  mazowieckie  warszawa         0.0     1790658    51724   
88789 2020-03-09  mazowieckie  warszawa         0.0     1790658    51724   
88790 2020-03-10  mazowieckie  warszawa         0.0     1790658    51724   
88791 2020-03-11  mazowieckie  warszawa         0.0     1790658    51724   
88792 2020-03-12  mazowieckie  warszawa         0.0     1790658    51724   
88793 2020-03-13  mazowieckie  warszawa         0.0     1790658    51724   
88794 2020-03-14  mazowieckie  warszawa         0.0     1790658    51724   
88795 2020-03-15  mazowieckie  warszawa         0.0     1790658    51724   
88796 2020-0

In [23]:
#posumowac dane z powiatów do województw
# last_province_name = ""
# for ind in population.index:
#     suma = 0
#     one_province = infections.loc[(infections['province'] == population['province'][ind]) & (infections['date'] == '2020-08-25')]
#     if not one_province.empty:
#         if not (one_province['province'].iloc[0] == last_province_name):
#             last_province_name = one_province['province'].iloc[0]
#             #print(one_province)
#             suma += one_province['population'].sum()
#             whole_province = one_province.loc[one_province['province'] == one_province['district']]
#             suma /= 2
#             print(str(one_province['province'].iloc[0]))
#             print("Z danych: "+ str(whole_province['population'].iloc[0]))
#             print("Posumowane: " + str(suma))
#             print()
        

In [24]:
print_full(population)

     population                 district             province
0       2891321             dolnośląskie         dolnośląskie
1         89762            bolesławiecki         dolnośląskie
2         99935           dzierżoniowski         dolnośląskie
3         88447                głogowski         dolnośląskie
4         34552                 górowski         dolnośląskie
5         49734                 jaworski         dolnośląskie
6         63333            jeleniogórski         dolnośląskie
7         42824           kamiennogórski         dolnośląskie
8        156283                  kłodzki         dolnośląskie
9         55110                 legnicki         dolnośląskie
10        53852                 lubański         dolnośląskie
11       105896                 lubiński         dolnośląskie
12        45599                 lwówecki         dolnośląskie
13        36800                  milicki         dolnośląskie
14       107268                oleśnicki         dolnośląskie
15      

In [25]:
# ['bieszczadzki', 'kazimierski', 'kolbuszowski', 'krosno', 'leski', 'leżajski', 'namysłowski', 'ostrołęka', 'tarnobrzeski]
# powiaty, w których dane się urywają na 30.12.2020